# Virtual SLAM and Navigation Using Gazebo

Using SLAM (short for Simultaneous Localization and Mapping) techniques, you will be able to execute autonomous navigation with GoPiGo3.

SLAM is a technique used in robotics to explore and map an unknown environment while estimating the pose of the robot itself. As it moves all around, it will be acquiring structured information of the surroundings by processing the raw data coming from its sensors.

For optimal and easy-to-understand coverage of the topic of SLAM, we will implement a 360º-coverage Laser Distance Sensor (LDS) in the virtual robot. There are low-cost versions of this sensor technology, such as EAI YDLIDAR X4 (available at https://www.aliexpress.com/item/32908156152.html), which is the one we will make use of in the next chapter.


Copy its files to the ROS workspace to have them available, and leave the rest outside of the src folder.


In [ ]:
cp -R ~/Hands-On-ROS-for-Robotics-Programming/Chapter8_Virtual_SLAM ~/rUBotCoop_ws/src/

The code contains two new ROS packages as follows: 
- gopigo3_description, which contains the URDF model plus the SDF (Gazebo tags) for a complete, dynamic simulation. This package provides the gopigo3_rviz.launch launch file to interactively visualize the model in RViz.
- virtual_slam contains the virtual robot simulation itself, plus the launch files needed to run SLAM in Gazebo.

Then, rebuild the workspace so that it is known to your ROS installation:

In [ ]:
cd ~/rUBotCoop_ws
catkin_make 

## ROS navigation packages

First, let's prepare your machine with the required ROS packages needed for the navigation stack:


In [ ]:
sudo apt install ros-melodic-navigation ros-melodic-amcl ros-melodic-map-server ros-melodic-move-base ros-melodic-urdf ros-melodic-xacro ros-melodic-compressed-image-transport ros-melodic-rqt-image-view

And finally the slam_gmapping package, that the time of writing is already available in its binary version:

In [ ]:
sudo apt-get install ros-melodic-slam-gmapping

## Adding sensors to the GoPiGo3 model


Up to now, you should have equipped your virtual robot with a differential drive controller that provides the capability to convert velocity commands into rotations of the left and right wheels. We need to complete the model with some sort of perception of the environment. For this, we will add controllers for two common sensors, a two-dimensional camera and an LDS. The first corresponds to the Pi camera of your physical robot, while the second is the unidirectional distance sensor of the GoPiGo3 kit.


### Camera model


You can add the solid of the camera as usual with <visual> tags, but since it is a commercial device, you can a get better look by using a realistic three-dimensional CAD model supplied by the manufacturer or made by someone else in the open source community. The URDF definition is as follows:


In [ ]:
<link name="camera">
  <visual>
    <origin xyz="0.25 0 0.05" rpy="0 1.570795 0" />
    <geometry>
      <mesh filename="package://virtual_slam/meshes/piCamera.stl" scale="0.5 0.5 0.5"/>
    </geometry>
  </visual>
...
</link>
<joint name="joint_camera" type="fixed">
    <parent link="base_link"/>
    <child link="camera"/>
    <origin xyz="0 0 0" rpy="0 0 0" /> 
    <axis xyz="1 0 0" />
</joint>

Then we add the camera technical features using a <gazebo> tag that emulates the behavior of the camera:

In [ ]:
<gazebo reference="camera">
  <sensor type="camera" name="camera1">
    <update_rate>30.0</update_rate>
    <camera name="front">
      <horizontal_fov>1.3962634</horizontal_fov>
      <image>
        <width>800</width>
        <height>800</height>
        <format>R8G8B8</format>
      </image>
    <clip>
      <near>0.02</near>
      <far>300</far>
    </clip>
    </camera>
    <!-- plugin "camera_controller" filename="libgazebo_ros_camera.so" -->
  </sensor>
</gazebo>


camera images will be published in the /gopigo/camera1/image_raw topic.

Launch the ROS visualization tool to check that the model is properly built. Since RViz only represents its visual features—it does not include any physical simulation engine—it is a much lighter environment than Gazebo and you have available all the options to check every aspect of the appearance of the model:

In [ ]:
roslaunch gopigo3_description gopigo3_basic_rviz.launch

<img src="./Images/08_model_sensors1.png">

Let's first place the robot in Gazebo the same way we did in the previous chapter and enable remote control with the keyboard:

In [ ]:
roslaunch virtual_slam gopigo3_basic_world.launch
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel

key_teleop allows you to remotely control the GoPiGo3 with the arrow keys of your keyboard.

Now, launch a node from the image_view package that comes preinstalled with ROS (we are remapping the image topic so that the node takes its data from the camera node topic, /gopigo/camera1/image_raw):

In [ ]:
rosrun image_view image_view image:=/gopigo/camera1/image_raw 

Teleoperate the robot with the arrow keys and you will see the subjective view in the image window:

<img src="./Images/08_Gazebo_Cam1.png">

let's obtain the ROS graph with the well-known command, rqt_graph, and have a look at how the topic remapping for the image is handled:

Thanks to the mapping argument, image:=/gopigo/camera1/image_raw, the image topic of the image_view package remains implicit and just the /gopigo/camera1/image_raw is visible.

<img src="./Images/08_Gazebo_rqt_cam.png">

## Distance sensor

We add the solid model of this sensor under the <visual> tag by following the same procedure we covered for the camera. The URDF definition is as follows:

In [ ]:
<joint name="distance_sensor_solid_joint" type="fixed">
    <axis xyz="0 1 0" />
    <origin rpy="0 0 0" xyz="0 0 0" />
    <parent link="base_link"/>
    <child link="distance_sensor_solid"/>
</joint>
<link name="distance_sensor_solid">
    <visual>
      <origin xyz="0.2 0 0.155" rpy="1.570795 0 1.570795" />
      <geometry>
        <mesh filename="package://gopigo3_description/meshes/IR_Sensor_Sharp_GP2Y_solid.stl" scale="0.005 0.005 0.005"/>
      </geometry>
      <material name="red"/>
    </visual>
    ...
</link>

Then we add the sensor technical features using a <gazebo> tag, which you can see refers to the distance_sensor link defined in the preceding snippet (not distance_sensor_solid):

In [ ]:
<gazebo reference="distance_sensor"> 
   <sensor type="ray" name="laser_distance">
      <visualize>true</visualize>
      <update_rate>10</update_rate>
      <ray>
         ...
         <range>
            <min>0.01</min>
            <max>3</max>
            <resolution>0.01</resolution>
         </range>
      </ray>
      <!-- plugin filename="libgazebo_ros_range.so" name="gazebo_ros_ir" -->
    </sensor> 
   </gazebo>

The <update_rate> tag specifies that the sensor is read at a frequency of 10 Hz, and the <range> tag sets measured distance values between 10 cm and 3 m at 1 cm resolution.

Finally, we add the Gazebo plugin that emulates the behavior of the distance sensor. The following snippet is what substitutes the commented line referring to plugin "gazebo_ros_ir" in the preceding code block:

In [ ]:
      <plugin filename="libgazebo_ros_range.so" name="gazebo_ros_ir">
         <gaussianNoise>0.005</gaussianNoise>
         <alwaysOn>true</alwaysOn>
         <updateRate>0.0</updateRate>
             <topicName>gopigo/distance_sensor</topicName>
             <frameName>distance_sensor</frameName>
         <radiation>INFRARED</radiation>
         <fov>0.02</fov>
       </plugin>

Launch the ROS visualization tool to check that the model is properly built. Since RViz only represents its visual features, it is a much lighter environment than Gazebo and you have available all the options to check every aspect of the appearance of the model:

In [ ]:
roslaunch gopigo3_description gopigo3_basic_rviz.launch

<img src="./Images/08_model_sensors1.png">

This test includes both the distance sensor and the two-dimensional camera. Run the example by using four Terminals, as indicated in the following code:

In [ ]:
roslaunch virtual_slam gopigo3_basic_world.launch
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel
rostopic echo /gopigo/distance
rosrun image_view image_view image:=/gopigo/camera1/image_raw

<img src="./Images/08_Gazebo_distance_sensor1.png">